# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f15400e1d90>
├── 'a' --> tensor([[ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821]])
└── 'x' --> <FastTreeValue 0x7f15400e1940>
    └── 'c' --> tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                        [ 1.4964, -0.3674,  1.7245,  1.6877],
                        [-1.4849, -1.1669, -0.4097, -0.7690]])

In [4]:
t.a

tensor([[ 1.5680, -1.0839,  0.5744],
        [ 0.9571,  0.0340,  0.3821]])

In [5]:
%timeit t.a

63.2 ns ± 0.0718 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f15400e1d90>
├── 'a' --> tensor([[ 1.1934,  0.1991, -0.4932],
│                   [-0.6064,  2.1471, -1.6042]])
└── 'x' --> <FastTreeValue 0x7f15400e1940>
    └── 'c' --> tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                        [ 1.4964, -0.3674,  1.7245,  1.6877],
                        [-1.4849, -1.1669, -0.4097, -0.7690]])

In [7]:
%timeit t.a = new_value

60.6 ns ± 0.0117 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5680, -1.0839,  0.5744],
               [ 0.9571,  0.0340,  0.3821]]),
    x: Batch(
           c: tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5680, -1.0839,  0.5744],
        [ 0.9571,  0.0340,  0.3821]])

In [11]:
%timeit b.a

69.5 ns ± 0.0272 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.6451, -0.9732, -0.6357],
               [ 0.2140, -1.8660, -0.5102]]),
    x: Batch(
           c: tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690]]),
       ),
)

In [13]:
%timeit b.a = new_value

490 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

829 ns ± 0.194 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.6 µs ± 33.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 472 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 1.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f148ffc15b0>
├── 'a' --> tensor([[[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]],
│           
│                   [[ 1.5680, -1.0839,  0.5744],
│                    [ 0.9571,  0.0340,  0.3821]]])
└── 'x' --> <FastTreeValue 0x7f148ffc1cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 104 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f148ffc1640>
├── 'a' --> tensor([[ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821],
│                   [ 1.5680, -1.0839,  0.5744],
│                   [ 0.9571,  0.0340,  0.3821]])
└── 'x' --> <FastTreeValue 0x7f148ffc17f0>
    └── 'c' --> tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                        [ 1.4964, -0.3674,  1.7245,  1.6877],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 70.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.9 µs ± 174 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.7008,  0.4813,  1.7871,  1.2788],
                       [ 1.4964, -0.3674,  1.7245,  1.6877],
                       [-1.4849, -1.1669, -0.4097, -0.7690]],
              
                      [[ 0.7008,  0.4813,  1.7871,  1.2788],
                       [ 1.4964, -0.3674,  1.7245,  1.6877],
                       [-1.4849, -1.1669, -0.4097, -0.7690]],
              
                      [[ 0.7008,  0.4813,  1.7871,  1.2788],
                       [ 1.4964, -0.3674,  1.7245,  1.6877],
                       [-1.4849, -1.1669, -0.4097, -0.7690]],
              
                      [[ 0.7008,  0.4813,  1.7871,  1.2788],
                       [ 1.4964, -0.3674,  1.7245,  1.6877],
                       [-1.4849, -1.1669, -0.4097, -0.7690]],
              
                      [[ 0.7008,  0.4813,  1.7871,  1.2788],
                       [ 1.4964, -0.3674,  1.7245,  1.6877],
                       [-1.4849, -1.1669, -0.4097, -0.7690]],

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 156 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690],
                      [ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690],
                      [ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690],
                      [ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690],
                      [ 0.7008,  0.4813,  1.7871,  1.2788],
                      [ 1.4964, -0.3674,  1.7245,  1.6877],
                      [-1.4849, -1.1669, -0.4097, -0.7690],
                      [ 0.7008,  0.4813,  1.7871,  1.2788],
                   

In [28]:
%timeit Batch.cat(batches)

160 µs ± 368 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

309 µs ± 1.86 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
